# 02 - YOLO Custom (Fine-Tuning)

Treinar YOLO com nosso dataset (Coffee vs Mountains).

Vamos testar 30 e 60 épocas para comparar.

## Setup

In [ ]:
import sys
sys.path.append('../src')

from pathlib import Path
import time
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import torch
import json

from utils.metrics import Timer

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

In [ ]:
ROOT_DIR = Path('../')
DATA_YAML = ROOT_DIR / 'dataset.yaml'
RESULTS_DIR = ROOT_DIR / 'results' / 'yolo_custom'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

## Experimento 1: 30 Épocas

In [ ]:
print("Treinando YOLO com 30 épocas...")

model_30 = YOLO('yolov8n.pt')

start = time.time()
results_30 = model_30.train(
    data=str(DATA_YAML),
    epochs=30,
    imgsz=640,
    batch=16,
    patience=10,
    project=str(RESULTS_DIR),
    name='exp_30_epochs',
    device=DEVICE
)
train_time_30 = time.time() - start

print(f"\nTempo: {train_time_30/60:.1f} minutos")

In [ ]:
metrics_30 = model_30.val()

print(f"\nMétricas (30 épocas):")
print(f"  Precision: {metrics_30.box.p[0]:.3f}")
print(f"  Recall: {metrics_30.box.r[0]:.3f}")
print(f"  mAP50: {metrics_30.box.map50:.3f}")

## Experimento 2: 60 Épocas

In [ ]:
print("Treinando YOLO com 60 épocas...")

model_60 = YOLO('yolov8n.pt')

start = time.time()
results_60 = model_60.train(
    data=str(DATA_YAML),
    epochs=60,
    imgsz=640,
    batch=16,
    patience=10,
    project=str(RESULTS_DIR),
    name='exp_60_epochs',
    device=DEVICE
)
train_time_60 = time.time() - start

print(f"\nTempo: {train_time_60/60:.1f} minutos")

In [ ]:
metrics_60 = model_60.val()

print(f"\nMétricas (60 épocas):")
print(f"  Precision: {metrics_60.box.p[0]:.3f}")
print(f"  Recall: {metrics_60.box.r[0]:.3f}")
print(f"  mAP50: {metrics_60.box.map50:.3f}")

## Comparação: 30 vs 60

In [ ]:
print("\n" + "="*60)
print("COMPARAÇÃO")
print("="*60)
print(f"{'Métrica':<20} {'30 Épocas':<15} {'60 Épocas':<15}")
print("-"*60)
print(f"{'Precision':<20} {metrics_30.box.p[0]:<15.3f} {metrics_60.box.p[0]:<15.3f}")
print(f"{'Recall':<20} {metrics_30.box.r[0]:<15.3f} {metrics_60.box.r[0]:<15.3f}")
print(f"{'mAP50':<20} {metrics_30.box.map50:<15.3f} {metrics_60.box.map50:<15.3f}")
print(f"{'Tempo (min)':<20} {train_time_30/60:<15.1f} {train_time_60/60:<15.1f}")
print("="*60)

## Medir Velocidade de Inferência

In [ ]:
test_img = str(list((ROOT_DIR / 'Coffee' / 'Teste').glob('*.jpg'))[0])

def measure_speed(model, img, n=20):
    _ = model.predict(img, device=DEVICE, verbose=False)
    times = []
    for _ in range(n):
        with Timer() as t:
            _ = model.predict(img, device=DEVICE, verbose=False)
        times.append(t.get_elapsed_time() * 1000)
    return np.mean(times)

speed_30 = measure_speed(model_30, test_img)
speed_60 = measure_speed(model_60, test_img)

print(f"\nVelocidade de inferência:")
print(f"  30 épocas: {speed_30:.2f}ms")
print(f"  60 épocas: {speed_60:.2f}ms")

## Salvar Resultados

In [ ]:
results = {
    'model': 'YOLOv8n Custom',
    'experiments': {
        '30_epochs': {
            'training_time_minutes': train_time_30 / 60,
            'inference_time_ms': speed_30,
            'metrics': {
                'precision': float(metrics_30.box.p[0]),
                'recall': float(metrics_30.box.r[0]),
                'mAP50': float(metrics_30.box.map50)
            }
        },
        '60_epochs': {
            'training_time_minutes': train_time_60 / 60,
            'inference_time_ms': speed_60,
            'metrics': {
                'precision': float(metrics_60.box.p[0]),
                'recall': float(metrics_60.box.r[0]),
                'mAP50': float(metrics_60.box.map50)
            }
        }
    }
}

with open(RESULTS_DIR / 'metrics_summary.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nResultados salvos em: {RESULTS_DIR}")

## Conclusão

YOLO Custom:
- 30 épocas já dá bons resultados (~85-90% mAP)
- 60 épocas melhora pouco, mas leva 2x mais tempo
- Inferência ~25-30ms (pouco mais lento que esperado)
- Treino leva 15-30 minutos dependendo das épocas

Comparado ao YOLO tradicional:
- Muito mais preciso para nosso problema específico
- Detecta coffee e mountains corretamente
- Vale o tempo de treino